In [14]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras import layers, losses, models, optimizers

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.preprocessing import MinMaxScaler

import vae as V

def data_scaling(data):             #스케일러, main
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    return data


def data_sliding(data, n_features, Time_window): #데이터 밀기, main
    # reshape_num = data.shape[0] - Time_window + 1
    data = sliding_window_view(data, (Time_window, n_features))
    #   data = data.reshape(reshape_num, Time_window, n_features) <-필요없음

    return data


def data_load_processing(FILENAME, Time_window=None): #데이터 로딩 및 스케일러 사용, main
    df = pd.read_csv(FILENAME)
    df = df.drop(['Time'], axis=1)
    n_features = df.shape[1]
    x_train, x_test = train_test_split(df, test_size=0.9, shuffle=False)
    #x_validation, x_test = train_test_split(x_test, test_size=0.5, shuffle=False)

    x_train = data_scaling(x_train)
    x_test = data_scaling(x_test)
    #x_validation = data_scaling(x_validation)

    x_train = data_sliding(x_train, n_features, Time_window)
    x_test = data_sliding(x_test, n_features, Time_window)
    #x_validation = data_sliding(x_validation, n_features, Time_window)

    return x_train, x_test #, x_validation


def sampling(inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


def vae_model(input_shape, latent_dim, kernel_size, strides, padding):
    inputs = tf.keras.Input(shape=input_shape)
    h = layers.Conv2D(32, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv1", data_format="channels_first")(inputs)
    h = layers.Conv2D(64, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv2", data_format="channels_first")(h)
    h = layers.Conv2D(128, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv3", data_format="channels_first")(h)
    shape_save = h.get_shape().as_list()
    h = layers.Flatten()(h)
    h = layers.Dense(16, activation="relu")(h)

    # vae
    z_mean = layers.Dense(latent_dim, name='z_mean')(h)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(h)
    z = sampling([z_mean, z_log_var])
    encoder = tf.keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # decoder
    latent_inputs = tf.keras.Input(shape=(latent_dim,))
    h = layers.Dense(shape_save[1] * shape_save[2] * shape_save[3], activation="relu")(latent_inputs)
    h = layers.Reshape((shape_save[1], shape_save[2], shape_save[3]))(h)
    h = layers.Conv2DTranspose(64, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                               data_format="channels_first")(h)
    h = layers.Conv2DTranspose(32, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                               data_format="channels_first")(h)
    decoder_outputs = layers.Conv2DTranspose(1,kernel_size=kernel_size, strides=strides, padding=padding, activation="relu",
                                             data_format="channels_first")(h)
    decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")

    return encoder, decoder

def lstm_preprocess(latent_z, seq_length):
    reshape_num = latent_z.shape[0] - seq_length + 1
    slide_z = sliding_window_view(latent_z, (seq_length, latent_z.shape[1]))
    slide_z = slide_z.reshape(reshape_num, seq_length, latent_z.shape[1])
    slide_z = slide_z[:-1]

    dataY = []
    for i in range(0, len(latent_z) - seq_length):
        _y = latent_z[i + seq_length]  # 다음 나타날 z(정답)
        dataY.append(_y)  # dataY 리스트에 추가
    dataY = np.array(dataY)
    
    return slide_z, dataY
    
    
def lstm_model(seq_length, latent_dim, lstm_z):
    
    inputs = tf.keras.Input(shape=(seq_length, latent_dim))
    h = layers.LSTM(32, activation="relu", name='lstm1')(inputs)
    lstm_outputs = layers.Dense(latent_dim, activation='sigmoid')(h)
    lstm = tf.keras.Model(inputs, lstm_outputs, name='lstm')
    lstm.compile(loss='mse', optimizer=tf.keras.optimizers.Adam())
    lstm.fit(lstm_z, dataY, epochs=5, batch_size=256)
    lstm_results = lstm.predict(lstm_z)

    return lstm_results
    
    
if __name__ == '__main__':
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=config)

    #---------------------------------사용자 설정 변수---------------------------------------
    FILENAME = "BTC_USDT.csv"
    Time_window = 8
    latent_dim = 3
    #-------------------------------------------------------------------------------------

    #데이터 로딩
    x_train, x_test= data_load_processing(FILENAME, Time_window=Time_window)

    #VAE_model
    input_shape = (1, 8, 54)
    kernel_size = (2,4)
    strides = (2,2)
    padding = "valid"
    encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)

    #VAE
    vae = V.VAE(encoder, decoder)
    vae.compile(optimizer=tf.keras.optimizers.Adam())
    vae.fit(x_train, epochs=1, batch_size=128)

    z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)
    #------------------------------------------------------------------------------------------
    seq_length = 8
    
    lstm_z, dataY = lstm_preprocess(latent_z, seq_length)
    
    lstm_results = lstm_model(seq_length, latent_dim, lstm_z)

    decoder_output = vae.decoder.predict(lstm_results)

1233/1233 [==============================] - 9s 7ms/step - loss: 4.5889 - reconstruction_loss: 3.9925 - kl_loss: 0.0025
Epoch 1/5
617/617 [==============================] - 11s 17ms/step - loss: 1.0631
Epoch 2/5
617/617 [==============================] - 10s 17ms/step - loss: 0.9974
Epoch 3/5
617/617 [==============================] - 11s 17ms/step - loss: 0.9978
Epoch 4/5
617/617 [==============================] - 10s 17ms/step - loss: 0.9969
Epoch 5/5
617/617 [==============================] - 10s 17ms/step - loss: 0.9959


In [15]:
latent_z.shape

(157772, 3)

In [16]:
decoded_latent_z = vae.decoder.predict(latent_z)

In [17]:
x_train[0][0][0]

array([2.92028172e-01, 2.90334511e-01, 2.94325406e-01, 2.91690876e-01,
       1.28034781e-02, 2.91970143e-01, 1.70846712e-02, 2.91319238e-01,
       2.01296969e-02, 4.47492299e-01, 2.36533311e-01, 3.66674647e-01,
       3.92146587e-01, 2.91061150e-01, 2.12365833e-02, 2.91134819e-01,
       2.79011400e-02, 3.60360194e-01, 1.27683809e-01, 3.65558665e-01,
       2.11723599e-01, 2.91124649e-01, 3.42660671e-02, 2.90319558e-01,
       2.98881262e-02, 3.48278646e-01, 6.07620474e-02, 3.35166460e-01,
       1.17714478e-01, 2.85898890e-01, 2.37780000e-02, 2.85868523e-01,
       3.21074999e-02, 2.99450752e-01, 5.12985089e-02, 2.97449067e-01,
       6.24664374e-02, 2.82898658e-01, 5.55299442e-02, 2.84263507e-01,
       4.31056906e-02, 3.02565338e-01, 2.62291760e-02, 3.02324704e-01,
       4.48243776e-02, 5.33293333e-01, 3.61499602e-02, 9.99928230e-01,
       5.65686608e-01, 4.23772617e-04, 4.13801998e-01, 2.49842430e-01,
       9.89829098e-01, 3.75898567e-02])

In [18]:
decoded_latent_z[0][0][0]

array([0.4697462 , 0.46533424, 0.46264365, 0.4733061 , 0.02557664,
       0.46049118, 0.04326943, 0.459072  , 0.04562262, 0.47607002,
       0.22299135, 0.39032698, 0.35610285, 0.44361228, 0.0548588 ,
       0.45651662, 0.06094082, 0.39741763, 0.11658681, 0.39166713,
       0.19601147, 0.45313987, 0.06523179, 0.45492646, 0.07420598,
       0.39229473, 0.06616436, 0.38756633, 0.11082893, 0.45245504,
       0.08279014, 0.4342288 , 0.07797877, 0.31175783, 0.04195606,
       0.3161421 , 0.06201148, 0.4365373 , 0.09397884, 0.45480105,
       0.11531775, 0.30187708, 0.03229275, 0.3216901 , 0.04647939,
       0.5410715 , 0.02583014, 0.9865462 , 0.583456  , 0.        ,
       0.47923243, 0.49801818, 0.5026664 , 0.05862983], dtype=float32)

In [19]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

52.71626097927271

In [47]:
#VAE_model
input_shape = (1, 8, 54)
kernel_size = (2,4)
strides = (2,2)
padding = "valid"
encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)
encoder.summary()

#VAE
vae = V.VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(x_train, epochs=1, batch_size=128)

z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)

decoded_latent_z = vae.decoder.predict(latent_z)

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(None, 1, 8, 54)]   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 4, 25)    416         input_44[0][0]                   
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 2, 10)    24640       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 128, 1, 3)    98432       conv2[0][0]                      
____________________________________________________________________________________________

ValueError: in user code:

    C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\User\AppDa

In [46]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_37 (Dense)             (None, 512)               1536      
_________________________________________________________________
reshape_17 (Reshape)         (None, 128, 1, 4)         0         
_________________________________________________________________
conv2d_transpose_51 (Conv2DT (None, 64, 2, 11)         81984     
_________________________________________________________________
conv2d_transpose_52 (Conv2DT (None, 32, 4, 25)         20512     
_________________________________________________________________
conv2d_transpose_53 (Conv2DT (None, 1, 8, 53)          321       
Total params: 104,353
Trainable params: 104,353
Non-trainable params: 0
_____________________________________________________

In [2]:
x_train

array([[[[0.29202817, 0.29033451, 0.29432541, ..., 0.24984243,
          0.9898291 , 0.03758986],
         [0.29179521, 0.28975501, 0.2896421 , ..., 0.23856524,
          0.98941222, 0.03773672],
         [0.28779247, 0.28632038, 0.2896162 , ..., 0.26149706,
          0.98955855, 0.04105991],
         ...,
         [0.28733831, 0.2855053 , 0.28986814, ..., 0.24248091,
          0.98935997, 0.03957154],
         [0.28755951, 0.28550766, 0.29008947, ..., 0.25108924,
          0.98928226, 0.03705845],
         [0.28828663, 0.28737102, 0.29037909, ..., 0.36926206,
          0.98947052, 0.03338668]]],


       [[[0.29179521, 0.28975501, 0.2896421 , ..., 0.23856524,
          0.98941222, 0.03773672],
         [0.28779247, 0.28632038, 0.2896162 , ..., 0.26149706,
          0.98955855, 0.04105991],
         [0.28786306, 0.28551237, 0.28979044, ..., 0.16087802,
          0.98941098, 0.04397803],
         ...,
         [0.28755951, 0.28550766, 0.29008947, ..., 0.25108924,
          0.98928226, 0

In [6]:
decoded_latent_z = vae.decoder.predict(latent_z)

In [4]:
decoded_latent_z

array([[[[0.42212987, 0.41830298, 0.42025995, ..., 0.48775578,
          0.47784787, 0.04895516],
         [0.42028517, 0.41765738, 0.4194188 , ..., 0.4932936 ,
          0.47851002, 0.04948027],
         [0.42274404, 0.41937497, 0.42343286, ..., 0.4874905 ,
          0.47792765, 0.04955728],
         ...,
         [0.42161852, 0.42079133, 0.4182538 , ..., 0.49036297,
          0.4767546 , 0.05024262],
         [0.42297825, 0.41901863, 0.42302048, ..., 0.4868034 ,
          0.47816405, 0.05073044],
         [0.42358723, 0.41765776, 0.42346552, ..., 0.49240515,
          0.47860187, 0.05065444]]],


       [[[0.42212987, 0.41830298, 0.42025995, ..., 0.48775578,
          0.47784787, 0.04895516],
         [0.42028517, 0.41765738, 0.4194188 , ..., 0.4932936 ,
          0.47851002, 0.04948027],
         [0.42274404, 0.41937497, 0.42343286, ..., 0.4874905 ,
          0.47792765, 0.04955728],
         ...,
         [0.42161852, 0.42079133, 0.4182538 , ..., 0.49036297,
          0.4767546 , 0

In [15]:
x_train[0][0][0]

array([2.92028172e-01, 2.90334511e-01, 2.94325406e-01, 2.91690876e-01,
       1.28034781e-02, 2.91970143e-01, 1.70846712e-02, 2.91319238e-01,
       2.01296969e-02, 4.47492299e-01, 2.36533311e-01, 3.66674647e-01,
       3.92146587e-01, 2.91061150e-01, 2.12365833e-02, 2.91134819e-01,
       2.79011400e-02, 3.60360194e-01, 1.27683809e-01, 3.65558665e-01,
       2.11723599e-01, 2.91124649e-01, 3.42660671e-02, 2.90319558e-01,
       2.98881262e-02, 3.48278646e-01, 6.07620474e-02, 3.35166460e-01,
       1.17714478e-01, 2.85898890e-01, 2.37780000e-02, 2.85868523e-01,
       3.21074999e-02, 2.99450752e-01, 5.12985089e-02, 2.97449067e-01,
       6.24664374e-02, 2.82898658e-01, 5.55299442e-02, 2.84263507e-01,
       4.31056906e-02, 3.02565338e-01, 2.62291760e-02, 3.02324704e-01,
       4.48243776e-02, 5.33293333e-01, 3.61499602e-02, 9.99928230e-01,
       5.65686608e-01, 4.23772617e-04, 4.13801998e-01, 2.49842430e-01,
       9.89829098e-01, 3.75898567e-02])

In [16]:
decoded_latent_z[0][0][0]

array([4.2266017e-01, 4.1910651e-01, 4.2137834e-01, 4.1884026e-01,
       2.2563705e-02, 4.0976927e-01, 3.4648228e-02, 3.9964333e-01,
       4.1503254e-02, 4.4699097e-01, 2.1279044e-01, 3.7215990e-01,
       3.2540226e-01, 4.0850085e-01, 4.4138357e-02, 4.0126529e-01,
       5.6429986e-02, 3.6651522e-01, 1.1869589e-01, 3.7555310e-01,
       1.8580799e-01, 4.0370101e-01, 5.9828606e-02, 4.0133244e-01,
       6.1059427e-02, 3.6953765e-01, 6.7552850e-02, 3.4580654e-01,
       1.0494846e-01, 3.9960480e-01, 6.9483466e-02, 3.9598104e-01,
       7.5246632e-02, 3.0019993e-01, 4.1288666e-02, 2.9332805e-01,
       5.5803001e-02, 4.0018663e-01, 9.5556520e-02, 3.9629942e-01,
       9.8795488e-02, 2.9242331e-01, 3.1640388e-02, 2.9613435e-01,
       4.2429965e-02, 5.0648785e-01, 2.2149986e-02, 9.6365142e-01,
       5.7560956e-01, 4.6942648e-04, 4.6373710e-01, 4.8748574e-01,
       4.7824594e-01, 4.9235519e-02], dtype=float32)

In [18]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

40.11670439928918